In [ ]:
# 📚 Celda 1: Configurar la raíz del proyecto automáticamente usando pyprojroot

# Asegúrate de tener pyprojroot instalado primero:
# pip install pyprojroot

import pyprojroot
import sys
import os
from typing import Dict
import shutil



# 🔥 Detectar automáticamente la raíz del proyecto
project_root = pyprojroot.here()

# 🔥 Insertar la raíz en sys.path para permitir imports como si estuvieras en t-p/
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"✅ Proyecto raíz detectado dinámicamente: {project_root}")

# 🔥 Ahora puedes importar setup_path como un módulo normal
import setup_path

# (setup_path ejecutará su propio proceso de detección interna también)


In [ ]:
# 📚 Celda 2: Leer el contenido del archivo .env desde la raíz real detectada

def read_env_file(filepath: str) -> list:
    """Lee un archivo .env y devuelve las líneas crudas (incluyendo comentarios)."""
    if not os.path.exists(filepath):
        print(f"⚠️ Archivo {filepath} no encontrado. Se creará uno nuevo al guardar.")
        return []
    
    with open(filepath, "r", encoding="utf-8") as file:
        lines = file.readlines()
    
    return lines

# 🔥 Definir la ruta real del archivo .env (ya estamos en project_root)
env_path = os.path.join(project_root, ".env")
print(f"📄 Ruta completa al archivo .env detectada: {env_path}")

# 🔥 Leer el contenido del .env
env_lines = read_env_file(env_path)

# 🔥 Mostrar el contenido leído
if env_lines:
    print("\n📄 Contenido actual del .env:")
    for line in env_lines:
        print(line.strip())
else:
    print("\n📄 El archivo .env está vacío o no existe.")


In [ ]:
# 📚 Celda 3: Parsear las variables del archivo .env a un diccionario

def parse_env_variables(lines: list) -> Dict[str, str]:
    """Convierte las líneas del .env en un diccionario de variables."""
    env_vars = {}
    
    for line in lines:
        stripped = line.strip()
        
        # Ignorar líneas vacías o comentarios
        if not stripped or stripped.startswith("#"):
            continue
        
        # Procesar líneas tipo VARIABLE=valor
        if "=" in stripped:
            key, value = stripped.split("=", 1)
            env_vars[key.strip()] = value.strip()
    
    return env_vars

# 🔥 Parsear el contenido leído
env_variables = parse_env_variables(env_lines)

# 🔥 Mostrar las variables encontradas
if env_variables:
    print("\n🧩 Variables detectadas en el .env:")
    for key, value in env_variables.items():
        print(f"  {key} = {value}")
else:
    print("\n🧩 No se detectaron variables en el .env.")


In [ ]:
# 📚 Celda 4: Buscar y validar las credenciales de MySQL en el .env

# 🔥 Variables que necesitamos para MySQL
mysql_required_keys = [
    "MYSQL_HOST",
    "MYSQL_PORT",
    "MYSQL_USER",
    "MYSQL_PASSWORD",
    "MYSQL_DATABASE"  # Opcional, pero lo tratamos como requerido aquí
]

# 🔥 Función para validar si tenemos todas las credenciales
def check_mysql_credentials(env_vars: Dict[str, str], required_keys: list) -> Dict[str, str]:
    """Verifica si las credenciales requeridas para MySQL están presentes."""
    missing_keys = []
    found_credentials = {}
    
    for key in required_keys:
        value = env_vars.get(key)
        if value is None or value.strip() == "":
            missing_keys.append(key)
        else:
            found_credentials[key] = value
    
    return {"found": found_credentials, "missing": missing_keys}

# 🔥 Ejecutar la validación
mysql_credentials_status = check_mysql_credentials(env_variables, mysql_required_keys)

# 🔥 Mostrar resultados
print("\n📋 Resultado de verificación de credenciales MySQL:")

if mysql_credentials_status["found"]:
    print("\n✅ Credenciales encontradas:")
    for key, value in mysql_credentials_status["found"].items():
        print(f"  {key} = {value}")

if mysql_credentials_status["missing"]:
    print("\n⚠️ Credenciales faltantes:")
    for key in mysql_credentials_status["missing"]:
        print(f"  - {key}")
else:
    print("\n🎯 Todas las credenciales requeridas están presentes.")


In [ ]:
# 📚 Celda 5: Solicitar al usuario las credenciales faltantes de MySQL

def request_mysql_credentials(missing_keys: list) -> Dict[str, str]:
    """Solicita al usuario las credenciales faltantes de MySQL."""
    new_credentials = {}
    
    print("\n🛠️ Solicitud de credenciales de MySQL:")

    for key in missing_keys:
        # Definir preguntas amigables
        if key == "MYSQL_HOST":
            prompt = "Host de MySQL (default 127.0.0.1): "
            value = input(prompt).strip() or "127.0.0.1"
        elif key == "MYSQL_PORT":
            prompt = "Puerto de MySQL (default 3306): "
            value = input(prompt).strip() or "3306"
        elif key == "MYSQL_USER":
            prompt = "Usuario de MySQL (default root): "
            value = input(prompt).strip() or "root"
        elif key == "MYSQL_PASSWORD":
            prompt = "Contraseña de MySQL: "
            value = input(prompt).strip()
        elif key == "MYSQL_DATABASE":
            prompt = "Nombre de la base de datos (puede ser vacío si no aplica): "
            value = input(prompt).strip()
        else:
            prompt = f"{key}: "
            value = input(prompt).strip()

        new_credentials[key] = value
    
    return new_credentials

# 🔥 Solicitar las credenciales faltantes
new_mysql_credentials = request_mysql_credentials(mysql_credentials_status["missing"])

# 🔥 Combinar credenciales encontradas + nuevas
final_mysql_credentials = {**mysql_credentials_status["found"], **new_mysql_credentials}

# 🔥 Mostrar resumen final
print("\n📋 Credenciales finales de MySQL para usar:")

for key, value in final_mysql_credentials.items():
    display_value = value if key != "MYSQL_PASSWORD" else "********"  # Ocultar password
    print(f"  {key} = {display_value}")


In [ ]:
# 📚 Celda 6: Guardar/actualizar las credenciales de MySQL en el archivo .env

def update_env_variables(lines: list, updates: Dict[str, str]) -> list:
    """
    Actualiza las variables en las líneas de un archivo .env.
    Si una variable existe, la reemplaza. Si no, agrega un comentario y luego las nuevas variables.
    """
    updated_lines = []
    existing_keys = set()
    
    for line in lines:
        stripped = line.strip()
        
        if not stripped or stripped.startswith("#") or "=" not in stripped:
            updated_lines.append(line)
            continue
        
        key, _ = stripped.split("=", 1)
        key = key.strip()
        
        if key in updates:
            new_value = updates[key]
            updated_lines.append(f"{key}={new_value}\n")
            existing_keys.add(key)
        else:
            updated_lines.append(line)
    
    # 🔥 Aquí, DENTRO de la función:
    new_variables_to_add = [key for key in updates.keys() if key not in existing_keys]
    if new_variables_to_add:
        updated_lines.append("\n# Información de la base de datos encontrada\n")
    
    # 🔥 Agregar las nuevas variables que no existían
    for key, value in updates.items():
        if key not in existing_keys:
            updated_lines.append(f"{key}={value}\n")
    
    return updated_lines

def save_env_file(filepath: str, lines: list):
    """Guarda el archivo .env actualizado."""
    # Crear backup de seguridad primero
    backup_path = filepath + ".bak"
    shutil.copy(filepath, backup_path)
    print(f"📄 Backup del .env guardado en: {backup_path}")

    # Guardar el nuevo contenido
    with open(filepath, "w", encoding="utf-8") as f:
        f.writelines(lines)
    print(f"✅ Archivo .env actualizado exitosamente.")

# 🔥 Actualizar las líneas originales con las nuevas credenciales
updated_env_lines = update_env_variables(env_lines, final_mysql_credentials)

# 🔥 Guardar las líneas actualizadas en el .env
save_env_file(env_path, updated_env_lines)


In [ ]:
# 📚 Celda 7: Probar conexión MySQL e imprimir información básica del servidor

def test_mysql_connection(credentials: Dict[str, str]):
    """Prueba la conexión a MySQL y muestra información básica del servidor."""
    try:
        import mysql.connector
        
        # Preparar configuración
        connection = mysql.connector.connect(
            host=credentials.get("MYSQL_HOST", "127.0.0.1"),
            port=int(credentials.get("MYSQL_PORT", 3306)),
            user=credentials.get("MYSQL_USER", "root"),
            password=credentials.get("MYSQL_PASSWORD", ""),
            database=credentials.get("MYSQL_DATABASE", None) or None  # Puede ser vacío
        )
        
        if connection.is_connected():
            print("✅ Conexión exitosa a MySQL.\n")
            
            cursor = connection.cursor()
            
            # 🔵 Mostrar versión de MySQL
            cursor.execute("SELECT VERSION()")
            version = cursor.fetchone()[0]
            print(f"🛠️ Versión de MySQL: {version}")
            
            # 🔵 Mostrar base de datos actual (si hay)
            cursor.execute("SELECT DATABASE()")
            current_db = cursor.fetchone()[0]
            print(f"📚 Base de datos actual seleccionada: {current_db if current_db else 'Ninguna'}")
            
            # 🔵 Listar todas las bases de datos
            print("\n📋 Bases de datos disponibles:")
            cursor.execute("SHOW DATABASES")
            databases = cursor.fetchall()
            for db in databases:
                print(f"  - {db[0]}")
            
            cursor.close()
        
        connection.close()
        
    except Exception as e:
        print(f"❌ Error al conectar o consultar MySQL: {str(e)}")

# 🔥 Ejecutar la prueba de conexión usando las credenciales finales
test_mysql_connection(final_mysql_credentials)
